## Examples from Data Analysis with Python and PySpark

(A.K.A PySpark in Action)

In [1]:
import numpy as np
import pandas as pd
import pyspark
import urllib

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = (SparkSession
         .builder
         .appName("Analyzing vocabulary")
         .getOrCreate())

# getOrCreate avoids creation of a new session, if one exists 
# but this may mean you can't change some JVM config options

## Read Data

In [4]:
url = "https://www.gutenberg.org/files/205/205-0.txt"
urllib.request.urlretrieve(url, './thoreau.txt')

('./thoreau.txt', <http.client.HTTPMessage at 0x406f5da650>)

In [3]:
book = spark.read.text("./thoreau.txt")

book.printSchema()

root
 |-- value: string (nullable = true)



In [8]:
# See docs
!pyspark spark.read?

Type:        property
String form: <property object at 0x406f810720>
Docstring:  
Returns a :class:`DataFrameReader` that can be used to read data
in as a :class:`DataFrame`.

.. versionadded:: 2.0.0

Returns
-------
:class:`DataFrameReader`


In [11]:
book.show(10, truncate=50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|The Project Gutenberg eBook of Walden, by Henry...|
|                                                  |
|This eBook is for the use of anyone anywhere in...|
|most other parts of the world at no cost and wi...|
|whatsoever. You may copy it, give it away or re...|
|of the Project Gutenberg License included with ...|
|www.gutenberg.org. If you are not located in th...|
|will have to check the laws of the country wher...|
|                                 using this eBook.|
|                                                  |
+--------------------------------------------------+
only showing top 10 rows



## Tokenization

In [4]:
# split the string by spaces, and change the name of the column
# then select that one column
lines = book.select(F.split("value", " ").alias("line"))

lines.show(5, truncate=50)

+--------------------------------------------------+
|                                              line|
+--------------------------------------------------+
|[The, Project, Gutenberg, eBook, of, Walden,, b...|
|                                                []|
|[This, eBook, is, for, the, use, of, anyone, an...|
|[most, other, parts, of, the, world, at, no, co...|
|[whatsoever., You, may, copy, it,, give, it, aw...|
+--------------------------------------------------+
only showing top 5 rows



In [15]:
book.select(book.value)

DataFrame[value: string]

In [18]:
book.select(book["value"])

DataFrame[value: string]

In [19]:
book.select(col("value"))

DataFrame[value: string]

In [16]:
book.select("value")

DataFrame[value: string]

In [5]:
# explode turns a column of vectors into a column, like ravel() in numpy

words = lines.select(F.explode("line").alias("word"))

words.show(15)

+---------+
|     word|
+---------+
|      The|
|  Project|
|Gutenberg|
|    eBook|
|       of|
|  Walden,|
|       by|
|    Henry|
|    David|
|  Thoreau|
|         |
|     This|
|    eBook|
|       is|
|      for|
+---------+
only showing top 15 rows



## Cleaning Data

In [6]:
words_lower = words.select(F.lower("word").alias("word_lower"))
words_lower.show()

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|   walden,|
|        by|
|     henry|
|     david|
|   thoreau|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
|  anywhere|
+----------+
only showing top 20 rows



In [7]:
words_clean = words_lower.select(
    F.regexp_extract("word_lower", "[a-z]+", 0).alias("word")
)

words_clean.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|   walden|
|       by|
|    henry|
|    david|
|  thoreau|
|         |
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
+---------+
only showing top 20 rows



In [9]:
# You can use ~ operator to invert the filter expression
words_nonull = words_clean.filter(F.col("word") != "")

words_nonull.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|   walden|
|       by|
|    henry|
|    david|
|  thoreau|
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
|       in|
+---------+
only showing top 20 rows



## Count

Spark groups records similar to pandas, allowing us to group all occurences of the same word, and count them up with an aggregation function

In [10]:
groups = words_nonull.groupby("word")

In [11]:
results = groups.count()
results.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  363|
|        those|  137|
|          few|   79|
|        still|  173|
|       poetry|   13|
|          art|   23|
|        trail|    5|
|    arguments|    3|
|    solemnity|    1|
|          fog|    4|
|       travel|   10|
|gratification|    1|
|       spared|    2|
|        cures|    1|
|      elevate|    1|
|       marrow|    2|
|    recognize|    8|
|         hope|    8|
|   strawberry|    2|
+-------------+-----+
only showing top 20 rows



In [13]:
# Small exercise to return the counts of different lengths of word

words_nonull.select(F.length(F.col("word")).alias('length')).groupby("length").count().show()

+------+-----+
|length|count|
+------+-----+
|    12|  539|
|     1| 5186|
|    13|  282|
|     6| 9272|
|    16|    3|
|     3|27123|
|     5|13954|
|    15|   27|
|     9| 3429|
|    17|    2|
|     4|21585|
|     8| 4535|
|     7| 7436|
|    10| 2075|
|    11| 1053|
|    14|   77|
|     2|22062|
+------+-----+



## Report

In [14]:
 results.orderBy("count", ascending=False).show()

+-----+-----+
| word|count|
+-----+-----+
|  the| 7518|
|  and| 4632|
|   of| 3615|
|   to| 3199|
|    a| 3093|
|   in| 2120|
|    i| 1991|
|   it| 1728|
|   is| 1353|
| that| 1340|
|   as| 1223|
|  not| 1073|
|  for|  993|
|   or|  956|
| with|  925|
|  was|  887|
|which|  871|
|  but|  805|
|   my|  779|
|   he|  765|
+-----+-----+
only showing top 20 rows



In [17]:
!rm -r count_report
!rm -r single_report

rm: cannot remove 'count_report': No such file or directory


In [18]:
# Notice what actually gets written
# We get an output file per partition of the data
results.write.csv("./count_report")

In [19]:
!ls ./count_report/*

./count_report/part-00000-35aad9c7-5f75-4e17-b0fc-cb2828deaf97-c000.csv
./count_report/_SUCCESS


In [20]:
# Put all the data on a single partition
results.coalesce(1).write.csv("./single_report")

In [21]:
!ls ./single_report/*

./single_report/part-00000-46b642e5-5990-4fc4-89b5-a4d1980e7b99-c000.csv
./single_report/_SUCCESS


## All together

In [22]:
# Note that you could replace most of the F.col() calls with just the string
# except: "word" != "" will fail in the .where() call

report = (
    spark.read.text("./thoreau.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.col("word") != "")
    .groupby("word")
    .count()
)           

Jobs can be submitted also via spark-submit - convert this to a python script and run:

In [ ]:
!spark-submit exercise_04.py